In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import os


INPUT_DIR = os.path.join( os.getcwd(), "mininet")
data = pd.DataFrame()
dirs = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR))]    
data = {}
client_data = pd.DataFrame()
for file in dirs:
    temp =  pd.read_csv(os.path.join(INPUT_DIR, file), sep=',')
    temp.columns= ['timestamp','gateway','latency','sender']
    client_data = pd.concat([client_data,temp])
    
    
client_data.set_index(pd.DatetimeIndex(client_data['timestamp']), inplace=True)
client_data.index = client_data.index.map(lambda x: x.replace(second=0, microsecond=0))
client_data.sort_index(inplace=True)
print(client_data)


#data[client] = client_data
#print(data)

                               timestamp    gateway  latency     sender
timestamp                                                              
2019-04-01 11:57:00  2019-04-01 11:57:14   10.0.1.8    0.147   10.0.0.9
2019-04-01 11:57:00  2019-04-01 11:57:15   10.0.1.4    0.192   10.0.0.7
2019-04-01 11:57:00  2019-04-01 11:57:14   10.0.1.4    0.155   10.0.0.8
2019-04-01 11:57:00  2019-04-01 11:57:14   10.0.1.3    0.463  10.0.0.10
2019-04-01 11:57:00  2019-04-01 11:57:15   10.0.1.4    0.157   10.0.0.3
2019-04-01 11:57:00  2019-04-01 11:57:15   10.0.1.8    0.262   10.0.0.1
2019-04-01 11:57:00  2019-04-01 11:57:15   10.0.1.7    0.223   10.0.0.4
2019-04-01 11:57:00  2019-04-01 11:57:15   10.0.1.2    0.221   10.0.0.6
2019-04-01 11:57:00  2019-04-01 11:57:15   10.0.1.1    0.145   10.0.0.2
2019-04-01 11:57:00  2019-04-01 11:57:14   10.0.1.2    0.460   10.0.0.5
2019-04-01 11:59:00  2019-04-01 11:59:14  10.0.1.10    0.063   10.0.0.7
2019-04-01 11:59:00  2019-04-01 11:59:15   10.0.1.5    0.159   1

In [2]:
def get_min_date():
    min_date = None
    for d in data:
        c = data.get(d)
        if min_date is None or min_date > c.index.min():
            min_date = c.index.min()
    return min_date#.strftime("%m/%d/%Y, %H:%M:%S")

def get_max_date():
    max_date = None
    for d in data:
        c = data.get(d)
        if max_date is None or max_date > c.index.max():
            max_date = c.index.max()
    return max_date

def get_90th_percentile_mean():
    df = pd.DataFrame()
    for d in data:
        df = pd.concat([df,data.get(d)])
        
    #print(df.describe())
    #print(df['latency'].max())
    #print(df['latency'].min())
    #print("Mean:",df['latency'].mean())
    v = df['latency'].quantile(0.9)
    df = df.query('latency<'+str(v))
    #print("90th percentile:"+str(v)+"Average:"+str(df['latency'].mean()))
    return df['latency'].mean()

def send_info(ts):
    min_date = None
    max_date = None
    
    gateway_t = pd.DataFrame()
    for d in data:
        c = data.get(d)[ts]
        if len(c)>2:
            c = c.sample(n=2)
            c["sender"] = d
            gateway_t = pd.concat([gateway_t , c])
        elif len(c)>0:
            c["sender"] = d
            gateway_t = pd.concat([gateway_t, c])
    return gateway_t     
        

In [101]:
import math

def calculate_trust_score(trust_score, gateway_table, node):
    
    for name, grouped in gateway_table.groupby('sender'):
        if name == node:
            continue
        mean_value = 0.0
        for index, row in grouped.iterrows():
            temp_gw = gateway_table[gateway_table.sender !=name]
            temp = temp_gw[temp_gw.gateway.isin([row['gateway']])]
            avg = temp['latency'].mean()   
            if math.isnan(avg):
                avg= 0.0
            mean_value += abs(row['latency'] - avg)
            
        mean_value = mean_value/len(grouped)
        if mean_value == 0.0:
            continue
        
        if name in trust_score:
            trust_score[name] = 2/3*mean_value + 1/3*trust_score[name]
        else:
            trust_score[name] = mean_value
    
    return trust_score

def build_gateway_table(gateway_table, new_df, trust_score, ts, topK):
    trusted_col = dict(sorted(trust_score.items(),key=lambda kv: kv[1])[:K])
    print(trusted_col)
    new_df = new_df[new_df.sender.isin(trusted_col.keys())]
    df_period = new_df.groupby(new_df['gateway'])
    actual = []
    gw_t = [] 
    for name, group in df_period:
        cnt = 1
        size = len(group)
        
        mv_value = 0
        if name in gateway_table:
            cnt +=1
            size +=1
            mv_value +=gateway_table[name]
        
        summa = sum(range(1,size+1))
        for index, row in group.iterrows():
            mv_value += cnt*row['latency']
            cnt +=1
        
        mv_value = mv_value/summa
        #print(name,mv_value)
        
        df_actual = data['c0_1'].query('timestamp == "'+ts+'" and gateway == "'+name+'"')
        if len(df_actual) >0:
            actual.append(df_actual['latency'].values[0])
            gw_t.append(mv_value)
    if len(actual) == 0 or len(gw_t) == 0:
        return None
    return cosine_similarity(actual, gw_t)

In [104]:
trust_score = {}
gateway_table = {}

for name, group in client_data.groupby(level=0):
    trust_score = calculate_trust_score(trust_score, group, '10.0.0.1')
    trust_score = dict(sorted(trust_score.items(),key=lambda kv: kv[1]))
    #print(name, trust_score)


2019-04-01 11:57:00 {'10.0.0.3': 0.016499999999999987, '10.0.0.8': 0.01949999999999999, '10.0.0.7': 0.036000000000000004, '10.0.0.9': 0.11500000000000002, '10.0.0.2': 0.145, '10.0.0.4': 0.223, '10.0.0.5': 0.23900000000000002, '10.0.0.6': 0.23900000000000002, '10.0.0.10': 0.46299999999999997}
2019-04-01 11:59:00 {'10.0.0.8': 0.04461111111111111, '10.0.0.3': 0.048166666666666656, '10.0.0.7': 0.04894444444444444, '10.0.0.9': 0.09938888888888889, '10.0.0.6': 0.1165, '10.0.0.2': 0.135, '10.0.0.5': 0.14988888888888888, '10.0.0.4': 0.17688888888888887, '10.0.0.10': 0.1902222222222222}
2019-04-01 12:01:00 {'10.0.0.7': 0.05387037037037036, '10.0.0.2': 0.06658333333333333, '10.0.0.8': 0.06987037037037037, '10.0.0.5': 0.07671296296296296, '10.0.0.3': 0.07705555555555554, '10.0.0.9': 0.08107407407407408, '10.0.0.10': 0.08382407407407406, '10.0.0.6': 0.10916666666666666, '10.0.0.4': 0.11512962962962962}
2019-04-01 12:03:00 {'10.0.0.3': 0.034518518518518504, '10.0.0.10': 0.03594135802469135, '10.0.0

In [ ]:
print(gateway_table)